In [1]:
import os
import pandas as pd
from IPython.display import display
from collections import Counter
from scipy.stats import chi2_contingency

In [2]:
path_to_data = "./data/"

In [3]:
def path_join(filename: str, path_to_folder: str = path_to_data) -> str:
    """
    os.path.join with default path_to_data.
    
    :param str filename: filename.
    :param str path_to_folder: path to the folder where the filename is located.
    :return: joined path.
    :rtype: str
    """
    
    return os.path.join(path_to_folder, filename)

### columns description

In [4]:
df_desc = pd.read_csv(path_join('HomeCredit_columns_description.csv'), encoding='cp1251', index_col=0)

In [5]:
df_desc[df_desc.Table == 'application_{train|test}.csv'].head()

,Table,Row,Description,Special
1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
2,application_{train|test}.csv,TARGET,Target variable (1 - client with payment diffi...,NaN
5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN


### load train

In [6]:
df_application_train = pd.read_csv(path_join('application_train.csv'))

In [7]:
df_application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df_application_train.tail()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
307510,456255,0,Cash loans,F,N,N,0,157500.0,675000.0,49117.5,...,0,0,0,0,0.0,0.0,0.0,2.0,0.0,1.0


In [9]:
df_application_train.shape

(307511, 122)

In [10]:
# get columns discription for each column in train
for column in df_application_train.columns:
    column_description = df_desc[df_desc.Row == column].Description.values
    
    print(f"{column}: {column_description}")
    print(116*'=')

SK_ID_CURR: ['ID of loan in our sample'
 'ID of loan in our sample - one loan in our sample can have 0,1,2 or more related previous credits in credit bureau '
 'ID of loan in our sample' 'ID of loan in our sample'
 'ID of loan in our sample' 'ID of loan in our sample']
TARGET: ['Target variable (1 - client with payment difficulties: he/she had late payment more than X days on at least one of the first Y installments of the loan in our sample, 0 - all other cases)']
NAME_CONTRACT_TYPE: ['Identification if loan is cash or revolving'
 'Contract product type (Cash loan, consumer loan [POS] ,...) of the previous application']
CODE_GENDER: ['Gender of the client']
FLAG_OWN_CAR: ['Flag if the client owns a car']
FLAG_OWN_REALTY: ['Flag if client owns a house or flat']
CNT_CHILDREN: ['Number of children the client has']
AMT_INCOME_TOTAL: ['Income of the client']
AMT_CREDIT: ['Credit amount of the loan'
 'Final credit amount on the previous application. This differs from AMT_APPLICATION in a wa

In [11]:
# get nunique for each column in train
for column in df_application_train.columns:
    column_nunique = df_application_train[column].nunique()
    column_5_unique = df_application_train[column].unique()[:5]
    
    print(f"{column}: {column_nunique}")
    print(f"{column}: {column_5_unique}")
    print(116*'=')

SK_ID_CURR: 307511
SK_ID_CURR: [100002 100003 100004 100006 100007]
TARGET: 2
TARGET: [1 0]
NAME_CONTRACT_TYPE: 2
NAME_CONTRACT_TYPE: ['Cash loans' 'Revolving loans']
CODE_GENDER: 3
CODE_GENDER: ['M' 'F' 'XNA']
FLAG_OWN_CAR: 2
FLAG_OWN_CAR: ['N' 'Y']
FLAG_OWN_REALTY: 2
FLAG_OWN_REALTY: ['Y' 'N']
CNT_CHILDREN: 15
CNT_CHILDREN: [0 1 2 3 4]
AMT_INCOME_TOTAL: 2548
AMT_INCOME_TOTAL: [202500. 270000.  67500. 135000. 121500.]
AMT_CREDIT: 5603
AMT_CREDIT: [ 406597.5 1293502.5  135000.   312682.5  513000. ]
AMT_ANNUITY: 13672
AMT_ANNUITY: [24700.5 35698.5  6750.  29686.5 21865.5]
AMT_GOODS_PRICE: 1002
AMT_GOODS_PRICE: [ 351000. 1129500.  135000.  297000.  513000.]
NAME_TYPE_SUITE: 7
NAME_TYPE_SUITE: ['Unaccompanied' 'Family' 'Spouse, partner' 'Children' 'Other_A']
NAME_INCOME_TYPE: 8
NAME_INCOME_TYPE: ['Working' 'State servant' 'Commercial associate' 'Pensioner' 'Unemployed']
NAME_EDUCATION_TYPE: 5
NAME_EDUCATION_TYPE: ['Secondary / secondary special' 'Higher education' 'Incomplete higher'
 'Lo

ELEVATORS_AVG: 257
ELEVATORS_AVG: [0.   0.08  nan 0.16 0.4 ]
ENTRANCES_AVG: 285
ENTRANCES_AVG: [0.069  0.0345    nan 0.2069 0.1379]
FLOORSMAX_AVG: 403
FLOORSMAX_AVG: [0.0833 0.2917    nan 0.1667 0.3333]
FLOORSMIN_AVG: 305
FLOORSMIN_AVG: [0.125  0.3333    nan 0.375  0.7083]
LANDAREA_AVG: 3527
LANDAREA_AVG: [0.0369 0.013     nan 0.0135 0.0931]
LIVINGAPARTMENTS_AVG: 1868
LIVINGAPARTMENTS_AVG: [0.0202 0.0773    nan 0.1202 0.2849]
LIVINGAREA_AVG: 5199
LIVINGAREA_AVG: [0.019  0.0549    nan 0.0778 0.1397]
NONLIVINGAPARTMENTS_AVG: 386
NONLIVINGAPARTMENTS_AVG: [0.     0.0039    nan 0.0193 0.0077]
NONLIVINGAREA_AVG: 3290
NONLIVINGAREA_AVG: [0.     0.0098    nan 0.1001 0.0044]
APARTMENTS_MODE: 760
APARTMENTS_MODE: [0.0252 0.0924    nan 0.084  0.1502]
BASEMENTAREA_MODE: 3841
BASEMENTAREA_MODE: [0.0383 0.0538    nan 0.101  0.1386]
YEARS_BEGINEXPLUATATION_MODE: 221
YEARS_BEGINEXPLUATATION_MODE: [0.9722 0.9851    nan 0.9811 0.9806]
YEARS_BUILD_MODE: 154
YEARS_BUILD_MODE: [0.6341 0.804     nan 0.7452 

FLAG_DOCUMENT_12: 2
FLAG_DOCUMENT_12: [0 1]
FLAG_DOCUMENT_13: 2
FLAG_DOCUMENT_13: [0 1]
FLAG_DOCUMENT_14: 2
FLAG_DOCUMENT_14: [0 1]
FLAG_DOCUMENT_15: 2
FLAG_DOCUMENT_15: [0 1]
FLAG_DOCUMENT_16: 2
FLAG_DOCUMENT_16: [0 1]
FLAG_DOCUMENT_17: 2
FLAG_DOCUMENT_17: [0 1]
FLAG_DOCUMENT_18: 2
FLAG_DOCUMENT_18: [0 1]
FLAG_DOCUMENT_19: 2
FLAG_DOCUMENT_19: [0 1]
FLAG_DOCUMENT_20: 2
FLAG_DOCUMENT_20: [0 1]
FLAG_DOCUMENT_21: 2
FLAG_DOCUMENT_21: [0 1]
AMT_REQ_CREDIT_BUREAU_HOUR: 5
AMT_REQ_CREDIT_BUREAU_HOUR: [ 0. nan  1.  2.  3.]
AMT_REQ_CREDIT_BUREAU_DAY: 9
AMT_REQ_CREDIT_BUREAU_DAY: [ 0. nan  1.  3.  2.]
AMT_REQ_CREDIT_BUREAU_WEEK: 9
AMT_REQ_CREDIT_BUREAU_WEEK: [ 0. nan  1.  3.  2.]
AMT_REQ_CREDIT_BUREAU_MON: 24
AMT_REQ_CREDIT_BUREAU_MON: [ 0. nan  1.  2.  6.]
AMT_REQ_CREDIT_BUREAU_QRT: 11
AMT_REQ_CREDIT_BUREAU_QRT: [ 0. nan  1.  2.  4.]
AMT_REQ_CREDIT_BUREAU_YEAR: 25
AMT_REQ_CREDIT_BUREAU_YEAR: [ 1.  0. nan  2.  4.]


### binary features

In [12]:
binary_columns = df_application_train.columns[df_application_train.nunique() == 2].tolist()

In [13]:
binary_columns

['TARGET',
 'NAME_CONTRACT_TYPE',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'EMERGENCYSTATE_MODE',
 'FLAG_DOCUMENT_2',
 'FLAG_DOCUMENT_3',
 'FLAG_DOCUMENT_4',
 'FLAG_DOCUMENT_5',
 'FLAG_DOCUMENT_6',
 'FLAG_DOCUMENT_7',
 'FLAG_DOCUMENT_8',
 'FLAG_DOCUMENT_9',
 'FLAG_DOCUMENT_10',
 'FLAG_DOCUMENT_11',
 'FLAG_DOCUMENT_12',
 'FLAG_DOCUMENT_13',
 'FLAG_DOCUMENT_14',
 'FLAG_DOCUMENT_15',
 'FLAG_DOCUMENT_16',
 'FLAG_DOCUMENT_17',
 'FLAG_DOCUMENT_18',
 'FLAG_DOCUMENT_19',
 'FLAG_DOCUMENT_20',
 'FLAG_DOCUMENT_21']

In [14]:
print(f"number of binary features: {len(binary_columns)}")

number of binary features: 37


### contingency matrix

In [15]:
# chi-square test of independence of variables in a contingency matrix
for column in binary_columns:
    
    target_crosstab = pd.crosstab(
        index=df_application_train[column],
        columns=df_application_train['TARGET'],
        margins=True,
    )
    display(target_crosstab)
    
    _, p_value, _, _ = chi2_contingency(target_crosstab)
    print(f"p-value: {p_value}\n")


TARGET,0,1,All
TARGET,,,
0,282686,0,282686
1,0,24825,24825
All,282686,24825,307511


p-value: 0.0



TARGET,0,1,All
NAME_CONTRACT_TYPE,,,
Cash loans,255011,23221,278232
Revolving loans,27675,1604,29279
All,282686,24825,307511


p-value: 2.6845456940854183e-62



TARGET,0,1,All
FLAG_OWN_CAR,,,
N,185675,17249,202924
Y,97011,7576,104587
All,282686,24825,307511


...

p-value: 0.9666913795720936



TARGET,0,1,All
FLAG_DOCUMENT_20,,,
0,282543,24812,307355
1,143,13,156
All,282686,24825,307511


p-value: 0.9999746780322938



TARGET,0,1,All
FLAG_DOCUMENT_21,,,
0,282597,24811,307408
1,89,14,103
All,282686,24825,307511


p-value: 0.37583962179951236



#### gender

In [16]:
Counter(df_application_train['CODE_GENDER'])

Counter({'M': 105059, 'F': 202448, 'XNA': 4})

In [17]:
# filter XNA
gender_filter = (df_application_train['CODE_GENDER'] != 'XNA')

In [18]:
target_crosstab = pd.crosstab(
    index=df_application_train['CODE_GENDER'][gender_filter],
    columns=df_application_train['TARGET'][gender_filter],
    margins=True,
)
target_crosstab

TARGET,0,1,All
CODE_GENDER,,,
F,188278,14170,202448
M,94404,10655,105059
All,282682,24825,307507


In [19]:
_, p_value, _, _ = chi2_contingency(target_crosstab)
print(f"p-value: {p_value}\n")

p-value: 6.241024167295911e-198

